Projeto de Inteligência Artifical voltado para o Reconhecimento da Datilologia

Grupo: Alina Tainá Germano Costa, Júlia de Oliveira Tavares, Lucas Emanuel Camilo Bezerra e Pedro Henrique Correia Bezerra Silva


A FAZER:
- CRIAR GRÁFICOS: MUITO IMPORTANTE PARA ENTENDER OS RESULTADOS DOS MODELOS
- DIVIDIR A PARTE DE VALIDAÇÃO
- USAR A BIBLIOTECA GRIDSEARCHCV
- RESPONDER O RELATÓRIO COM BASE NOS RESULTADOS OBTIDOS 
- APRESENTAR COM FUNDAMENTO SOBRE O PORQUÊ DAS NOSSAS ESCOLHAS

POSSIBILIDADE: 
- ALGORITMOS DE OTIMIZAÇÃO JUNTO DO GRIDSEARCHCV
- IDENTIFICAR O SINAL PELA WEBCAM AO VIVO

In [ ]:
# import de algumas bibliotecas essenciais: PIL, Image - faz o processo de conversão da imagem; Numpy - fundamental para a manipulação de dados; 
# Matplotlib.pyplot - essencial para a plotagem de gráficos; os - necessária para acessar as imagens dentro do diretório "datilologia"
# Importação das bibliotecas necessárias
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
import os

In [ ]:
# Carregar os dados preparados
x_treino = np.load('x_treino.npy')  # Carrega as imagens de treinamento
y_treino = np.load('y_treino.npy')  # Carrega os rótulos de treinamento
z_treino = np.load('z_treino.npy')

x_validacao = np.load('x_validacao.npy')  # Carrega as imagens de validação
y_validacao = np.load('y_validacao.npy')  # Carrega os rótulos de validação
z_validacao = np.load('z_validacao.npy')

x_teste = np.load('x_teste.npy')  # Carrega as imagens de teste
y_teste = np.load('y_teste.npy')  # Carrega os rótulos de teste
z_teste = np.load('z_teste.npy')

print(np.shape(x_treino))
print(np.shape(x_validacao))
print(np.shape(x_teste))

In [ ]:
def plotar_matriz_confusao(respostas, predicoes, rotulos):
    cm = confusion_matrix(respostas, predicoes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=rotulos, yticklabels=rotulos)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

def plotar_relatorio_classificacao(respostas, predicoes, rotulos):
    report = classification_report(respostas, predicoes, target_names=rotulos, output_dict=True)
    metrics = ['precision', 'recall', 'f1-score']
    data = {metric: [report[cls][metric] for cls in rotulos] for metric in metrics}
    x = np.arange(len(rotulos))

    plt.figure(figsize=(10, 6))
    for i, metric in enumerate(metrics):
        plt.bar(x + i * 0.2, data[metric], width=0.2, label=metric)

    plt.xticks(x + 0.2, rotulos)
    plt.ylabel('Scores')
    plt.title('Classification Metrics by Class')
    plt.legend()
    plt.show()

def plotar_curvas_roc(respostas, probabilidade, rotulos):
    plt.figure(figsize=(10, 8))
    for i, class_name in enumerate(rotulos):
        fpr, tpr, _ = roc_curve(respostas == i, probabilidade[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_name} (AUC = {roc_auc:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=0.5)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend(loc='lower right')
    plt.show()

# TREE padrão:
- criterion: "gini"
- splitter: "best"
- max_depth: None
- min_samples_split: 2
- min_samples_leaf: 1
- min_weight_fraction_leaf: 0
- max_features: None
- random_state: None
- max_leaf_nodes: None
- min_impurity_decrease: 0
- class_weight: None
- ccp_alpha: 0

In [ ]:
# Inicializar e treinar o modelo
arvore_padrao = DecisionTreeClassifier()
arvore_padrao.fit(x_treino, y_treino)

DecisionTreeClassifier()

In [ ]:
predicao_validacao_padrao = arvore_padrao.predict(x_validacao)
treino_acuracia_padrao = accuracy_score(y_validacao, predicao_validacao_padrao)
print("Acurácia (modelo inicial):", treino_acuracia_padrao)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_validacao, predicao_validacao_padrao))

Previsões: ['O' 'E' 'Q' ... 'I' 'O' 'M']
Acurácia do modelo: 0.947861155120265


In [ ]:
predicao_teste_padrao = arvore_padrao.predict(x_teste)
teste_acuracia_padrao = accuracy_score(y_teste, predicao_teste_padrao)
print("Acurácia (modelo inicial):", teste_acuracia_padrao)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_teste, predicao_teste_padrao))

In [ ]:
# Visualização dos resultados
plt.figure(figsize=(6, 4))
plt.bar(['Treinamento', 'Teste'], [treino_acuracia_padrao, teste_acuracia_padrao], color=['blue', 'orange'])
plt.ylim(0, 1)
plt.ylabel('Acurácia')
plt.title('Comparação de Acurácia no Treinamento e Teste')
plt.grid(axis='y')
plt.show()

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    arvore_padrao, x_treino, y_treino, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia no Treinamento', marker='o', color='blue')
plt.plot(train_sizes, test_mean, label='Acurácia no Teste', marker='o', color='orange')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.grid()
plt.show()

In [ ]:
train_error = 1 - treino_acuracia_padrao
test_error = 1 - teste_acuracia_padrao

plt.figure(figsize=(6, 4))
plt.bar(['Treinamento', 'Teste'], [train_error, test_error], color=['blue', 'orange'])
plt.ylim(0, 1)
plt.ylabel('Erro (1 - Acurácia)')
plt.title('Erro no Treinamento vs. Teste')
plt.grid(axis='y')
plt.show()

In [ ]:
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_validacao = arvore_padrao.predict_proba(x_validacao)

plotar_matriz_confusao(y_validacao, predicao_validacao_padrao, letras)
plotar_relatorio_classificacao(y_validacao, predicao_validacao_padrao, letras)
plotar_curvas_roc(y_validacao, y_prob_validacao, letras)

In [ ]:
# Código para gerar gráficos de métricas do modelo.
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_teste = arvore_padrao.predict_proba(x_teste)

# Código de exemplo (ajustar conforme as variáveis do notebook)
plotar_matriz_confusao(y_teste, predicao_teste_padrao, letras)
plotar_relatorio_classificacao(y_teste, predicao_teste_padrao, letras)
plotar_curvas_roc(y_teste, y_prob_teste, letras)

In [ ]:
x_validacao_ruidosa = x_validacao + np.random.normal(0, 0.1, x_validacao.shape)
x_validacao_ruidosa = np.clip(x_validacao_ruidosa, 0, 1)

y_pred_ruidosa = arvore_padrao.predict(x_validacao_ruidosa)
treino_acuracia_padrao = accuracy_score(y_validacao, y_pred_ruidosa)

print("Acurácia:", treino_acuracia_padrao)

In [ ]:
x_teste_ruidosa = x_teste + np.random.normal(0, 0.1, x_teste.shape)
x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1)

y_pred_ruidosa = arvore_padrao.predict(x_teste_ruidosa)
teste_acuracia_padrao = accuracy_score(y_teste, y_pred_ruidosa)

print("Acurácia:", teste_acuracia_padrao)

# TREE otimizado:
- criterion: 'entropy'
- max_depth: 10
- min_samples_leaf: 50
- min_samples_split: 10

In [ ]:
#Define os hiperparâmetros para a busca
parametros = {
    'criterion': ['entropy'],
    'max_depth': [ 10, 40],
    'min_samples_split': [10],
    'min_samples_leaf': [50]
}

#Inicializa o modelo de árvore de decisão
arvore_padrao = DecisionTreeClassifier(random_state=42)

#Configura o GridSearchCV
grid_search = GridSearchCV(
    estimator=arvore_padrao,
    param_grid=parametros,
    cv=2,  # Número de divisões para validação cruzada
    scoring='accuracy',  # Métrica para avaliar os modelos
    n_jobs=-1  # Utilizar todos os núcleos disponíveis
)

#Realiza o fit do GridSearchCV
grid_search.fit(x_treino, y_treino)

c:\Users\pitel\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=2, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [10, 40],
                         'min_samples_leaf': [50], 'min_samples_split': [10]},
             scoring='accuracy')

In [ ]:
# Exibir melhores parâmetros e melhor score
print("Melhores parâmetros encontrados:", grid_search.best_params_)
print("Melhor score encontrado:", grid_search.best_score_)

Melhores parâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 50, 'min_samples_split': 10}
Acurácia do melhor modelo no conjunto de teste: 0.8510730231888233


In [ ]:
# Inicializar e treinar o modelo
arvore_otimizada = grid_search.best_estimator_ 
arvore_otimizada.fit(x_treino, y_treino)

In [ ]:
predicao_validacao_otimizado = arvore_otimizada.predict(x_validacao)
treino_acuracia_otimizado = accuracy_score(y_validacao, predicao_validacao_otimizado)
print("Acurácia (modelo inicial):", treino_acuracia_otimizado)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_validacao, predicao_validacao_otimizado))

In [ ]:
predicao_teste_otimizado = arvore_otimizada.predict(x_teste)
teste_acuracia_otimizado = accuracy_score(y_teste, predicao_teste_otimizado)
print("Acurácia (modelo inicial):", teste_acuracia_otimizado)
print("Relatório de Classificação (modelo inicial):\n", classification_report(y_teste, predicao_teste_otimizado))

In [ ]:
# Visualização dos resultados
plt.figure(figsize=(6, 4))
plt.bar(['Treinamento', 'Teste'], [treino_acuracia_otimizado, teste_acuracia_otimizado], color=['blue', 'orange'])
plt.ylim(0, 1)
plt.ylabel('Acurácia')
plt.title('Comparação de Acurácia no Treinamento e Teste')
plt.grid(axis='y')
plt.show()

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(
    arvore_otimizada, x_treino, y_treino, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10)
)

train_mean = np.mean(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Acurácia no Treinamento', marker='o', color='blue')
plt.plot(train_sizes, test_mean, label='Acurácia no Teste', marker='o', color='orange')
plt.xlabel('Tamanho do Conjunto de Treinamento')
plt.ylabel('Acurácia')
plt.title('Curvas de Aprendizado')
plt.legend()
plt.grid()
plt.show()

In [ ]:
train_error = 1 - treino_acuracia_otimizado
test_error = 1 - teste_acuracia_otimizado

plt.figure(figsize=(6, 4))
plt.bar(['Treinamento', 'Teste'], [train_error, test_error], color=['blue', 'orange'])
plt.ylim(0, 1)
plt.ylabel('Erro (1 - Acurácia)')
plt.title('Erro no Treinamento vs. Teste')
plt.grid(axis='y')
plt.show()

In [ ]:
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_validacao = arvore_otimizada.predict_proba(x_validacao)

plotar_matriz_confusao(y_validacao, predicao_validacao_otimizado, letras)
plotar_relatorio_classificacao(y_validacao, predicao_validacao_otimizado, letras)
plotar_curvas_roc(y_validacao, y_prob_validacao, letras)

In [ ]:
# Código para gerar gráficos de métricas do modelo.
letras = ["A", "B", "C", "D", "E", "F", "G", "I", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "Y"]
y_prob_teste = arvore_otimizada.predict_proba(x_teste)

# Código de exemplo (ajustar conforme as variáveis do notebook)
plotar_matriz_confusao(y_teste, predicao_teste_otimizado, letras)
plotar_relatorio_classificacao(y_teste, predicao_teste_otimizado, letras)
plotar_curvas_roc(y_teste, y_prob_teste, letras)

In [ ]:
x_validacao_ruidosa = x_validacao + np.random.normal(0, 0.1, x_validacao.shape)
x_validacao_ruidosa = np.clip(x_validacao_ruidosa, 0, 1)

y_pred_ruidosa = arvore_otimizada.predict(x_validacao_ruidosa)
treino_acuracia_padrao = accuracy_score(y_validacao, y_pred_ruidosa)

print("Acurácia:", treino_acuracia_padrao)

In [ ]:
x_teste_ruidosa = x_teste + np.random.normal(0, 0.1, x_teste.shape)
x_teste_ruidosa = np.clip(x_teste_ruidosa, 0, 1)

y_pred_ruidosa = arvore_otimizada.predict(x_teste_ruidosa)
teste_acuracia_padrao = accuracy_score(y_teste, y_pred_ruidosa)

print("Acurácia:", teste_acuracia_padrao)